In [1]:
from tweet_sentiment_service.model import SentimentModel
from tweet_sentiment_service.inference import SentimentExtractor, SentimentID
from tokenizers import ByteLevelBPETokenizer
import pandas as pd
import numpy as np
from fastapi.testclient import TestClient
from tweet_sentiment_service.app import app

#### Test Model Class only

In [5]:
MAX_LEN = 96
PATH = './config/'

tokenizer = ByteLevelBPETokenizer(
    vocab=PATH+'vocab-roberta-base.json', 
    merges=PATH+'merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
)
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}

In [8]:
# Data preprocessing
test = pd.read_csv('./training/test.csv').fillna('')
tweet = test.loc[2, 'text'] #'Last session of the day  http://twitpic.com/67ezh'
sentiment = 'negative'
ct = 1
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')

# INPUT_IDS
text1 = " "+" ".join(tweet.split())
enc = tokenizer.encode(text1)
s_tok = sentiment_id[sentiment]           
input_ids[0,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
attention_mask[0,:len(enc.ids)+5] = 1

# Model Setup
#model = SentimentModel(weights_path="./weights_final.h5", config_path=PATH)

In [12]:
tokenizer.encode(text1).ids

[7306,
 478,
 4342,
 261,
 5150,
 5378,
 42134,
 13265,
 6,
 79,
 34,
 7,
 6602,
 69,
 138,
 6,
 215,
 10,
 9208,
 328]

In [ ]:
DISPLAY = 1
predictions = model.predict(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids
        )

prediction_start = np.zeros((1,MAX_LEN))
prediction_end = np.zeros((1,MAX_LEN))

prediction_start += predictions[0]
prediction_end += predictions[1]

start_idx = np.argmax(prediction_start)
end_idx = np.argmax(prediction_end)

if start_idx > end_idx:
    selected_text = tweet

text = " "+" ".join(tweet.split())
encoded_text = tokenizer.encode(text)
selected_text = tokenizer.decode(encoded_text.ids[start_idx-1:end_idx])

print(selected_text)

#### Test Inference Module

In [ ]:
PATH = './config/'
# Input
test = pd.read_csv('./training/test.csv').fillna('')
tweet = test.loc[2, 'text'] #'Last session of the day  http://twitpic.com/67ezh'
sentiment = 'negative'

# Inference
sentiment_extractor = SentimentExtractor(weights_path="./weights_final.h5", config_path=PATH)

# Extract Sentiment from Tweet
ans = sentiment_extractor.extract_sentiment(tweet=tweet, sentiment=sentiment)
print(ans)

#### Test API

In [2]:
client = TestClient(app=app)

In [3]:
from tweet_sentiment_service.app import SentimentRequest
data = {
    "tweet": "Recession hit Veronique Branquinho, she has to quit her company, such a shame!",
    "sentiment": "negative",
}
response = client.post(url="/sentiment", json=data)

2024-11-12 09:50:03.708786: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ./config/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


1/1 [==============================] - 6s 6s/step


In [4]:
response.json()

{'status': 'success',
 'selected_text': ' recession hit veronique branquinho, she has to quit her'}

In [7]:
params = {
    "tweet": "I went on a walk with my dog yesterday, and saw a homeless guy. So sad!",
    "sentiment": "negative",
}
response = client.get(url="/sentiment", params=params)

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ./config/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


1/1 [==============================] - 6s 6s/step


In [9]:
response.json()

{'selected_text': 'I went on a walk with my dog yesterday, and saw a homeless guy. So sad!'}

In [22]:
import tensorflow as tf
from transformers import *
import numpy as np


# Hard Code some parameters of model
MAX_LEN = 96
DISPLAY = 1

class SentimentModel:
    def __init__(self, weights_path: str, config_path: str):
        """
        Initializes the model and loads trained weights.

        Args:
            weights_path: Path to the trained weights for the model.
            config_path: Path to the configuration files for the RoBERTa base model.
        """
        self.max_len = MAX_LEN
        self.config_roberta_base = config_path + "config-roberta-base.json"
        self.pre_trained_roberta_weights = config_path + "pretrained-roberta-base.h5"

        self.model = self.build_model()
        self.model.load_weights(weights_path) 

    def build_model(self) -> tf.keras.Model: # type: ignore
        """
        Builds the roBERTa based model to extract sentiment from tweets.
        It uses a pre-trained roBERTa model and several convolutional and dropout layers
        to extract sentiment from the text.

        Returns:
            tf.keras.Model: Compiled keras model.
        """
        ids = tf.keras.layers.Input((self.max_len,), dtype=tf.int32)
        att = tf.keras.layers.Input((self.max_len), dtype=tf.int32)
        tok = tf.keras.layers.Input((self.max_len), dtype=tf.int32)

        config = RobertaConfig.from_pretrained(self.config_roberta_base)
        bert_model = TFRobertaModel.from_pretrained(self.pre_trained_roberta_weights, config=config)
        x = bert_model(ids, attention_mask=att, token_type_ids=tok)
        print(x)
        x1 = tf.keras.layers.Dropout(0.1)(x[0]) 
        x1 = tf.keras.layers.Conv1D(1,1)(x1)
        x1 = tf.keras.layers.Flatten()(x1)
        x1 = tf.keras.layers.Activation('softmax')(x1)


        x2 = tf.keras.layers.Dropout(0.1)(x[0]) 
        x2 = tf.keras.layers.Conv1D(1,1)(x2)
        x2 = tf.keras.layers.Flatten()(x2)
        x2 = tf.keras.layers.Activation('softmax')(x2)


        model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
        optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

        model.compile(loss='categorical_crossentropy', optimizer=optimizer)

        return model


    def predict(self, input_ids: np.ndarray, attention_mask: np.ndarray, token_type_ids: np.ndarray) -> str:
        """
        Runs inference. It recieves the tokenized inputs and returns information about the start
        and end indices of the sequence that represents the sentiment.

        Args:
            input_ids (np.ndarray): 
            attention_mask (np.ndarray): 
            token_type_ids (np.ndarray):

        Returns:
            str: Extracted sentiment of the tweet.
        """
        preds = self.model.predict([input_ids, attention_mask, token_type_ids], verbose=DISPLAY)

        return preds
        


In [23]:
PATH = './config/'
SentimentModel(weights_path="weights_final", config_path=PATH)

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ./config/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 96, 768) dtype=float32 (created by layer 'tf_roberta_model_1')>, pooler_output=<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_roberta_model_1')>, past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for weights_final

In [14]:
import pytest
import tensorflow as tf
from unittest.mock import MagicMock, patch
from tokenizers import Encoding
from tweet_sentiment_service.inference import SentimentExtractor, SentimentID
from tweet_sentiment_service.model import SentimentModel


MOCK_WEIGHTS_PATH = "/path/to/mock_weights.h5"
MOCK_CONFIG_PATH = "/path/to/mock_config/"


@pytest.fixture(scope="function")
def sentiment_model_mock(mocker):
    # Mock the build_model method to return a tf.keras.Model
    mocker.patch.object(SentimentModel, "build_model", return_value=MagicMock())
    # Mock the load_weights method of the model instance
    mocker.patch.object(tf.keras.Model, "load_weights", MagicMock())
    # Mock init
    mocker.patch.object(SentimentModel, "__init__", return_value=None)

    return SentimentModel(weights_path=MOCK_WEIGHTS_PATH, config_path=MOCK_CONFIG_PATH)

def test_initialization(mocker, sentiment_model_mock):
    # Given

    # Mock the build_model method to return a tf.keras.Model
    build_model_mock = mocker.patch.object(SentimentModel, "build_model", return_value=MagicMock())
    # Mock the load_weights method of the model instance
    load_weights_mock = mocker.patch.object(tf.keras.Model, "load_weights", MagicMock())
    # Mock tokenizer
    tokenizer_mock = mocker.patch("tweet_sentiment_service.inference.ByteLevelBPETokenizer", return_value=MagicMock())
    # Mock SentimentModel init
    model_init_mock = mocker.patch.object(SentimentModel, "__init__", return_value=None)
    
    # When

    sentiment_extractor = SentimentExtractor(weights_path=MOCK_WEIGHTS_PATH, config_path=MOCK_CONFIG_PATH)

    # Then

    tokenizer_mock.assert_called_with(
        vocab=f"{MOCK_CONFIG_PATH}vocab-roberta-base.json", 
        merges=f"{MOCK_CONFIG_PATH}merges-roberta-base.txt", 
        lowercase=True, 
        add_prefix_space=True
    )

    model_init_mock.assert_called_once_with(weights_path=MOCK_WEIGHTS_PATH, config_path=MOCK_CONFIG_PATH)
    print("success")


test_initialization(mocker, )

In [ ]:
def test_tokenize_and_mask(mocker):
    # Given
    tweet = "This is a happy test tweet."
    sentiment = "positive"
    build_model_mock = mocker.patch.object(SentimentModel, "build_model", return_value=MagicMock())
    load_weights_mock = mocker.patch.object(tf.keras.Model, "load_weights", MagicMock())
    tokenizer_mock = mocker.patch("tweet_sentiment_service.inference.ByteLevelBPETokenizer", return_value=MagicMock())
    tokenizer_mock = mocker.patch("tweet_sentiment_service.inference.ByteLevelBPETokenizer", return_value=MagicMock())
    tokenizer_mock.encode.ids.return_value = [10, 11, 12]
    sentiment_extractor = SentimentExtractor(MOCK_WEIGHTS_PATH, MOCK_CONFIG_PATH)

    # When
    input_ids, attention_mask, token_type_ids = sentiment_extractor.tokenize_and_mask(tweet, sentiment)

    # Then
    assert input_ids.shape == (1, sentiment_extractor.model.max_len)
    assert attention_mask.shape == (1, sentiment_extractor.model.max_len)
    assert token_type_ids.shape == (1, sentiment_extractor.model.max_len)

    assert input_ids[0, 0] == 0
    assert input_ids[0, 4] == SentimentID.POSITIVE.value